# Running normal CPU methods on multiple threads
to investigate how it scales with using multiple threads on one CPU with the built-in Threads.@threads annotaions

In [1]:
CODE_ROOT = pwd() * "/"

"/Users/robertstrauss/Desktop/MPAS_Ocean_Julia/"

In [2]:
include(CODE_ROOT * "mode_init/MPAS_Ocean.jl")

include(CODE_ROOT * "visualization.jl")

include(CODE_ROOT * "mode_forward/time_steppers.jl")

forward_euler_step! (generic function with 1 method)

In [3]:
using PyPlot, PyCall
animation  = pyimport("matplotlib.animation")
ipydisplay = pyimport("IPython.display")

PyObject <module 'IPython.display' from '/Users/robertstrauss/.julia/conda/3/lib/python3.10/site-packages/IPython/display.py'>

# Load Mesh From File

In [8]:
mpasOcean = MPAS_Ocean(CODE_ROOT * "InertiaGravityWaveMesh",
                "base_mesh.nc", "mesh.nc", periodicity="Periodic"
)
mpasOcean.nCells

10000

In [9]:
function calculate_normal_velocity_tendency_threaded!(mpasOcean::MPAS_Ocean)
    mpasOcean.normalVelocityTendency[:] .= 0

    Threads.@threads for iEdge in 1:mpasOcean.nEdges
        # gravity term: take gradient of sshCurrent across edge
        cell1Index, cell2Index = mpasOcean.cellsOnEdge[:,iEdge]
        
        if cell1Index !== 0 && cell2Index !== 0
            mpasOcean.normalVelocityTendency[iEdge] = mpasOcean.gravity * ( mpasOcean.sshCurrent[cell1Index] - mpasOcean.sshCurrent[cell2Index] ) / mpasOcean.dcEdge[iEdge]
        end
        
        # coriolis term
        for i in 1:mpasOcean.nEdgesOnEdge[iEdge]
            eoe = mpasOcean.edgesOnEdge[i,iEdge]
            
            mpasOcean.normalVelocityTendency[iEdge] += mpasOcean.weightsOnEdge[i,iEdge] * mpasOcean.normalVelocityCurrent[eoe] * mpasOcean.fEdge[eoe]
        end
    end
end

calculate_normal_velocity_tendency_threaded! (generic function with 1 method)

In [10]:
Threads.nthreads()

1

In [11]:
using BenchmarkTools

In [12]:
@benchmark calculate_normal_velocity_tendency_threaded!(mpasOcean)

BenchmarkTools.Trial: 3167 samples with 1 evaluation.
 Range (min … max):  1.461 ms …   3.874 ms  ┊ GC (min … max): 0.00% … 54.46%
 Time  (median):     1.483 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.576 ms ± 295.263 μs  ┊ GC (mean ± σ):  3.79% ±  9.13%

  █▆▄▃▃▃▂▁▁                                                   ▁
  ███████████▆▆▇▆▆▅▅▅▃▁▄▁▄▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄▇▇█▇██▇▆ █
  1.46 ms      Histogram: log(frequency) by time      2.83 ms <

 Memory estimate: 2.29 MiB, allocs estimate: 30008.

In [13]:
@benchmark calculate_normal_velocity_tendency!(mpasOcean)

BenchmarkTools.Trial: 3976 samples with 1 evaluation.
 Range (min … max):  1.222 ms …   6.320 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.238 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.256 ms ± 110.840 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▆██▆▅▅▄▆▅▄▄▄▃▃▃▂▂ ▁▁  ▁▁ ▁ ▁ ▁ ▁▁                           ▁
  ████████████████████▇█████████████▇▇▇▅▇▅▅▄▅▄▅▅▄▃▄▅▅▄▅▅▃▄▅▃▄ █
  1.22 ms      Histogram: log(frequency) by time      1.42 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.